## Pipeline Tutorial with HomoNN

### install

`Pipeline` is distributed along with [fate_client](https://pypi.org/project/fate-client/).

```bash
pip install fate_client
```

To use Pipeline, we need to first specify which `FATE Flow Service` to connect to. Once `fate_client` installed, one can find an cmd enterpoint name `pipeline`:

In [1]:
!pipeline --help

Usage: pipeline [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  config  pipeline config tool
  init     - DESCRIPTION: Pipeline Config Command.


Assume we have a `FATE Flow Service` in 127.0.0.1:9380(defaults in standalone), then exec

In [2]:
!pipeline init --ip fateflow --port 9380

Pipeline configuration succeeded.


### Hetero SecureBoost Example

 Before start a modeling task, data to be used should be uploaded. Please refer to this [guide](https://github.com/FederatedAI/FATE/blob/master/doc/tutorial/pipeline/pipeline_tutorial_upload.ipynb).

The `pipeline` package provides components to compose a `FATE pipeline`.

In [3]:
from pipeline.backend.pipeline import PipeLine
from pipeline.component import Reader, DataTransform, Intersection, HeteroSecureBoost, Evaluation
from pipeline.interface import Data

Make a `pipeline` instance:

    - initiator: 
        * role: guest
        * party: 9999
    - roles:
        * guest: 9999
        * host: 10000
    

In [16]:
pipeline = PipeLine() \
        .set_initiator(role='guest', party_id=8888) \
        .set_roles(guest=8888, host=10000)

Define a `Reader` to load data

In [17]:
reader_0 = Reader(name="reader_0")
# set guest parameter
reader_0.get_party_instance(role='guest', party_id=8888).component_param(
    table={"name": "breast_hetero_guest", "namespace": "experiment"})
# set host parameter
reader_0.get_party_instance(role='host', party_id=10000).component_param(
    table={"name": "breast_hetero_host", "namespace": "experiment"})

Add a `DataTransform` component to parse raw data into Data Instance

In [18]:
data_transform_0 = DataTransform(name="data_transform_0")
# set guest parameter
data_transform_0.get_party_instance(role='guest', party_id=8888).component_param(
    with_label=True)
data_transform_0.get_party_instance(role='host', party_id=[10000]).component_param(
    with_label=False)

Add a `Intersection` component to perform PSI for hetero-scenario

In [19]:
intersect_0 = Intersection(name="intersect_0")

Now, we define the `HeteroSecureBoost` component. The following parameters will be set for all parties involved.

In [20]:
hetero_secureboost_0 = HeteroSecureBoost(name="hetero_secureboost_0",
                                         num_trees=5,
                                         bin_num=16,
                                         task_type="classification",
                                         objective_param={"objective": "cross_entropy"},
                                         encrypt_param={"method": "iterativeAffine"},
                                         tree_param={"max_depth": 3})


To show the evaluation result, an "Evaluation" component is needed.

In [21]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

Add components to pipeline, in order of execution:

    - data_transform_0 comsume reader_0's output data
    - intersect_0 comsume data_transform_0's output data
    - hetero_secureboost_0 consume intersect_0's output data
    - evaluation_0 consume hetero_secureboost_0's prediciton result on training data

Then compile our pipeline to make it ready for submission.

In [22]:
pipeline.add_component(reader_0)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(intersect_0, data=Data(data=data_transform_0.output.data))
pipeline.add_component(hetero_secureboost_0, data=Data(train_data=intersect_0.output.data))
pipeline.add_component(evaluation_0, data=Data(data=hetero_secureboost_0.output.data))
pipeline.compile();


Now, submit(fit) our pipeline:

In [23]:
pipeline.fit()


2022-11-28 15:00:58.086 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:83 - Job id is 202211281500544478260

2022-11-28 15:00:58.094 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:98 - Job is still waiting, time elapse: 0:00:00
m2022-11-28 15:01:00.125 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:125 - 
2022-11-28 15:01:00.127 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:02
2022-11-28 15:01:01.143 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:03
2022-11-28 15:01:02.161 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:04
2022-11-28 15:01:03.177 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component reader_0, time elapse: 0:00:05
2022-11-28 15:01:04.193 | INFO     

2022-11-28 15:01:34.767 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:00:36
2022-11-28 15:01:35.782 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:00:37
2022-11-28 15:01:36.799 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:00:38
2022-11-28 15:01:37.817 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:00:39
2022-11-28 15:01:38.832 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:00:40
2022-11-28 15:01:39.853 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component hetero_secureboost_0, time elapse: 0:00:41
2022-11-28 15:01:40.868 | INFO     | pip

2022-11-28 15:02:11.418 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:01:13
2022-11-28 15:02:12.434 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:01:14
2022-11-28 15:02:13.450 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:01:15
2022-11-28 15:02:14.467 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:01:16
2022-11-28 15:02:15.483 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:01:17
2022-11-28 15:02:16.499 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:127 - Running component evaluation_0, time elapse: 0:01:18
2022-11-28 15:02:17.508 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_st

Once training is done, trained model may be used for prediction. Optionally, save the trained pipeline for future use.

In [ ]:
pipeline.dump("pipeline_saved.pkl");

First, deploy needed components from train pipeline

In [ ]:
pipeline = PipeLine.load_model_from_file('pipeline_saved.pkl')
pipeline.deploy_component([pipeline.data_transform_0, pipeline.intersect_0, pipeline.hetero_secureboost_0]);

Define new `Reader` components for reading prediction data

In [ ]:
reader_1 = Reader(name="reader_1")
reader_1.get_party_instance(role="guest", party_id=9999).component_param(table={"name": "breast_hetero_guest", "namespace": "experiment"})
reader_1.get_party_instance(role="host", party_id=10000).component_param(table={"name": "breast_hetero_host", "namespace": "experiment"})

Optionally, define new `Evaluation` component.

In [ ]:
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

Add components to predict pipeline in order of execution:

In [ ]:
predict_pipeline = PipeLine()
predict_pipeline.add_component(reader_1)\
                .add_component(pipeline, 
                               data=Data(predict_input={pipeline.data_transform_0.input.data: reader_1.output.data}))\
                .add_component(evaluation_0, data=Data(data=pipeline.hetero_secureboost_0.output.data));


Then, run prediction job

In [ ]:
predict_pipeline.predict()

For more demo on using pipeline to submit jobs, please refer to [pipeline demos](https://github.com/FederatedAI/FATE/tree/master/examples/pipeline/demo)